In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

# Load data

In [2]:
import org.um.feri.ears.algorithms.gp.GPAlgorithmExecutor
import org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData
import org.um.feri.ears.util.gp_stats.GPProgramSolutionSimple

var dataFile = "C:\\Users\\marko\\UnityProjects\\GenIATraP_refactor\\GeneralTrainingEnvironmentForMAS\\evo_monitor\\src\\data\\multiConfigurationPrograssData.ser"
var progressData = GPAlgorithmMultiConfigurationsProgressData.deserializeState(dataFile)
System.out.println("Progress data loaded from file: " + progressData)

System.out.println("Data size: " +progressData.multiConfigurationProgressData.size)

Progress data loaded from file: org.um.feri.ears.util.gp_stats.GPAlgorithmMultiConfigurationsProgressData@382116d8
Data size: 1


# Prepare helper classes & functions

In [3]:
class SelectedIndividual(
    var selectedConfiguration: Int,
    var selectedRun: Int,
    var selectedGeneration: Int,
    var bestWorstIndividual: Int, // 0 = best, 1 = worst
) {
    constructor() : this(0, 0, 0, 0)
}

In [4]:
import org.um.feri.ears.individual.representations.gp.IndividualMatchResult

fun sumIndividualValues(individualMatchResult: IndividualMatchResult, sumAbsValues: Boolean = false): Double {
    var sum = 0.0
    for (key in individualMatchResult.individualValues.keys) {
        if(sumAbsValues){
            sum += Math.abs(individualMatchResult.individualValues[key]!!)
        }
        else {
            sum += individualMatchResult.individualValues[key]!!
        }
    }

    return sum
}

In [5]:
import org.yaml.snakeyaml.util.Tuple

// Prepare data (best, worst and average individual rating per generation)
fun prepareData(selectedConfiguration: Int, selectedRun: Int, bestGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, worstGenIndividualsRating: MutableList<GPProgramSolutionSimple>?, avgGenIndividualRatings: MutableList<Double>?, bestGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, worstGenIndividualAvgFitnesses: MutableList<GPProgramSolutionSimple>?, avgGenIndividualAvgFitnesses: MutableList<Double>?){

    var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
    var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

    for (genProgressData in multiRunProgressData.gensProgressData) {
        var bestGenIndividualRating = genProgressData.population[0]
        var worstIndividualRating = genProgressData.population[0]
        var avgGenIndividualRating = 0.0

        var bestGenIndividualAvgFitness = genProgressData.population[0]
        var worstIndividualAvgFitness = genProgressData.population[0]
        var avgGenIndividualAvgFitness = 0.0

        for (solution in genProgressData.population) {
            if (solution.finalIndividualFitness.additionalValues["Rating"]!! < bestGenIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                bestGenIndividualRating = solution
            }

            if (solution.finalIndividualFitness.additionalValues["Rating"]!! > worstIndividualRating.finalIndividualFitness.additionalValues["Rating"]!!) {
                worstIndividualRating = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) < sumIndividualValues(bestGenIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                bestGenIndividualAvgFitness = solution
            }

            if(sumIndividualValues(solution.finalIndividualFitness.avgMatchResult) > sumIndividualValues(worstIndividualAvgFitness.finalIndividualFitness.avgMatchResult)) {
                worstIndividualAvgFitness = solution
            }

            avgGenIndividualRating += solution.finalIndividualFitness.additionalValues["Rating"]!!
            avgGenIndividualAvgFitness += sumIndividualValues(solution.finalIndividualFitness.avgMatchResult)
        }

        if (bestGenIndividualsRating != null) {
            bestGenIndividualsRating.add(bestGenIndividualRating)
        }
        if (worstGenIndividualsRating != null) {
            worstGenIndividualsRating.add(worstIndividualRating)
        }

        avgGenIndividualRating /= genProgressData.population.size
        if (avgGenIndividualRatings != null) {
            avgGenIndividualRatings.add(avgGenIndividualRating)
        }

        if (bestGenIndividualAvgFitnesses != null) {
            bestGenIndividualAvgFitnesses.add(bestGenIndividualAvgFitness)
        }
        if (worstGenIndividualAvgFitnesses != null) {
            worstGenIndividualAvgFitnesses.add(worstIndividualAvgFitness)
        }

        avgGenIndividualAvgFitness /= genProgressData.population.size
        if (avgGenIndividualAvgFitnesses != null) {
            avgGenIndividualAvgFitnesses.add(avgGenIndividualAvgFitness)
        }
    }
}

In [14]:
// TODO Move this to GPProgramSolutionSimple
fun calculateDiversity(solution1: GPProgramSolutionSimple, solution2: GPProgramSolutionSimple, w1: Double, w2: Double) : Double {
    var nodeCountDiversity = 0.0

    // Gather all different node count key
    val nodeCountKeys = mutableSetOf<String>()
    nodeCountKeys.addAll(solution1.nodeCounts.keys)
    nodeCountKeys.addAll(solution2.nodeCounts.keys)

    // Calculate node count diversity
    for (key in nodeCountKeys) {
        val nodeCount1 = solution1.nodeCounts[key] ?: 0
        val nodeCount2 = solution2.nodeCounts[key] ?: 0
        nodeCountDiversity += Math.abs(nodeCount1 - nodeCount2)
    }

    // Normalize node count diversity
    nodeCountDiversity /= (solution1.treeSize + solution2.treeSize)

    // Calculate avg fitness diversity
    var avgFitnessDiversity = 0.0

    // Gather all different avg fitness individual values key
    val avgFitnessKeys = mutableSetOf<String>()
    avgFitnessKeys.addAll(solution1.finalIndividualFitness.avgMatchResult.individualValues.keys)
    avgFitnessKeys.addAll(solution2.finalIndividualFitness.avgMatchResult.individualValues.keys)

    // Calculate avg fitness diversity
    for (key in avgFitnessKeys) {
        val avgFitness1 = solution1.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0
        val avgFitness2 = solution2.finalIndividualFitness.avgMatchResult.individualValues[key] ?: 0.0

        avgFitnessDiversity += Math.abs(avgFitness1 - avgFitness2)
    }

    // Normalize avg fitness diversity
    avgFitnessDiversity /= (Math.abs(sumIndividualValues(solution1.finalIndividualFitness.avgMatchResult, true)) + Math.abs(sumIndividualValues(solution2.finalIndividualFitness.avgMatchResult, true)))

    // Calculate final diversity
    return w1 * avgFitnessDiversity + w2 * nodeCountDiversity
}

# Visualize data (line plot) - Best, worst and average individual rating per generation

In [10]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

// Prepare data
val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualRatings = mutableListOf<Double>()

val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

var multiConfigurationProgressData = progressData.multiConfigurationProgressData[selectedConfiguration]
var multiRunProgressData = multiConfigurationProgressData.multiRunProgressData[selectedRun]

var lastPhaseName = "";
val generations = mutableListOf<String>()
for (genProgressData in multiRunProgressData.gensProgressData) {
    generations.add(genProgressData.generation.toString() + genProgressData.executionPhaseName.substring(genProgressData.executionPhaseName.length - 6, genProgressData.executionPhaseName.length))
}

// Display data
val dfRating = dataFrameOf(
    "Generation" to generations + generations + generations,
    "Rating" to bestGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + worstGenIndividualsRating.map { it.finalIndividualFitness.additionalValues["Rating"]!! } + avgGenIndividualRatings,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

val dfAvgFitness = dataFrameOf(
    "Generation" to generations + generations + generations,
    "AvgFitness" to bestGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + worstGenIndividualAvgFitnesses.map { sumIndividualValues(it.finalIndividualFitness.avgMatchResult) } + avgGenIndividualAvgFitnesses,
    "Individual" to List(generations.size) { "Best" } + List(generations.size) { "Worst" } + List(generations.size) { "Average" }
)

// The plot variable stores a plot with the temperature values for each city
val linePlotRating =
    letsPlot(dfRating.toMap()) { x = "Generation"; y = "Rating"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Generation Rating")

val linePlotAvgFitness =
    letsPlot(dfAvgFitness.toMap()) { x = "Generation"; y = "AvgFitness"; color = "Individual"} + ggsize(2000,800) + geomPoint(shape = 20) + geomLine() + ggtitle("Best Generation Average fitness")

val plots = listOf(linePlotRating, null, linePlotAvgFitness, null)
gggrid(plots, ncol = 2)

<path d="M0.6729689223802653 127.44306003852037 L0.6729689223802653 127.44306003852037 L4.037813534281591 123.74793138725909 L7.402658146182918 118.81670889406732 L10.767502758084245 121.45671770361255 L14.13234736998557 122.07896979133741 L17.497191981886896 118.90329950436558 L20.862036593788222 122.06439415450404 L24.226881205689548 114.7696598165002 L27.591725817590874 116.39308278301758 L30.9565704294922 116.39308278301758 L34.32141504139353 136.38150450048806 L37.686259653294854 147.9932920350773 L41.05110426519618 152.7299884612236 L44.415948877097506 145.18371861894377 L47.78079348899883 153.33536187960982 L51.14563810090016 152.2171116520492 L54.51048271280148 154.12807249172502 L57.87532732470281 152.26972315016965 L61.240171936604135 146.70030817241656 L64.60501654850546 146.70030817241656 L67.9698611604068 127.29923821749102 L71.33470577230811 140.4693940211487 L74.69955038420945 139.72093575213484 L78.06439499611076 149.16506473806047 L81.4292396080121 144.52088880442602 L84.79408421991342 149.4807678457834 L88.15892883181475 137.87655472921148 L91.52377344371607 150.63848352073438 L94.8886180556174 134.5890935384837 L98.25346266751872 142.25196847543052 L101.61830727942005 134.21798799843393 L104.98315189132137 142.0367458229377 L108.3479965032227 140.86533819325535 L111.71284111512404 136.5363467585426 L115.07768572702535 151.12234212034235 L118.44253033892669 142.63728116711283 L121.807374950828 145.64068803459017 L125.17221956272934 149.91053077076748 L128.53706417463067 153.13199216774413 L131.901908786532 153.13199216774413 L135.26675339843334 138.61397891360008 L138.63159801033464 141.88395752868783 L141.99644262223597 144.02853798569862 L145.3612872341373 150.9933211215469 L148.72613184603864 146.45461725285398 L152.09097645793995 150.77554977974825 L155.45582106984128 151.94484203142574 L158.8206656817426 152.4853108759633 L162.18551029364394 154.16708074451432 L165.55035490554528 151.54076047780043 L168.91519951744658 154.64571913711032 L172.2800441293479 151.66055569897475 L175.64488874124925 153.97374884247162 L179.00973335315058 152.14228868487785 L182.37457796505188 154.61985693505113 L185.73942257695322 151.19777922951508 L189.10426718885455 155.49862931296337 L192.46911180075588 152.8038765677928 L195.8339564126572 154.15024984177577 L199.19880102455852 151.31467092378276 L202.56364563645985 154.3403540864111 L205.9284902483612 154.43575763442956 L209.2933348602625 154.91599279622923 L212.65817947216382 151.17805162366992 L216.02302408406516 145.10308859795916 L219.3878686959665 143.65188810813612 L222.75271330786782 154.75957765227105 L226.11755791976913 151.9529964724444 L229.48240253167046 147.68687270727563 L232.8472471435718 150.276286451605 L236.21209175547312 154.10368764236915 L239.57693636737443 150.1302571312706 L242.94178097927576 154.53723777636193 L246.3066255911771 152.07957796273428 L249.67147020307843 154.652983072227 L253.03631481497973 151.30354130595728 L256.40115942688107 153.24940588271298 L259.7660040387824 152.6232096385791 L263.13084865068373 146.16902075079997 L266.49569326258506 151.8326451115688 L269.8605378744864 153.63892885900495 L273.22538248638773 152.00077672015382 L276.590227098289 154.20561815510257 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M0.6729689223802653 33.82158151314132 L0.6729689223802653 33.82158151314132 L4.037813534281591 37.78588519171898 L7.402658146182918 38.63996223737469 L10.767502758084245 37.39598690642617 L14.13234736998557 29.96808269921739 L17.497191981886896 31.30742745918429 L20.862036593788222 27.933785086159155 L24.226881205689548 31.286843466437155 L27.591725817590874 33.27460276338883 L30.9565704294922 40.816614554458894 L34.32141504139353 41.25211970715613 L37.686259653294854 37.00972229464168 L41.05110426519618 29.30360152509583 L44.415948877097506 29.198828899655922 L47.78079348899883 28.408933105047176 L51.14563810090016 27.138068590137088 L54.51048271280148 20.2073

# Visualize data (radar plot) - Comparison of individuals from different generations, runs & configurations

In [11]:
// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 70, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 82, 0)) // Individual at position

val maxValues = mutableMapOf<String, Double>()
maxValues["SectorExploration"] = 5.0
maxValues["PowerUp_Pickup_Health"] = 10.0
maxValues["PowerUp_Pickup_Ammo"] = 15.0
maxValues["PowerUp_Pickup_Shield"] = 5.0
maxValues["MissilesFired"] = 20.0
maxValues["MissilesFiredAccuracy"] = 50.0
maxValues["SurvivalBonus"] = 5.0
maxValues["OpponentTrackingBonus"] = 5.0
maxValues["OpponentDestroyedBonus"] = 500.0
maxValues["DamageTakenPenalty"] = 50.0
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualsRating[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualsRating[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

// Find all fitness keys
val fitnessKeys = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    for (key in individual.finalIndividualFitness.avgMatchResult.individualValues.keys) {
        if (key !in fitnessKeys) {
            fitnessKeys.add(key)
        }
    }
}

// Order fitnessKeys
fitnessKeys.sort()

// Build labels for visualization
val labels = mutableListOf<String>()
for (individual in selectedGPAlgorithmSolutionsSimple) {
    labels.addAll(fitnessKeys)
}

val datasets = mutableListOf<MutableList<Double>>()

// Build datasets for visualization
for (individual in selectedGPAlgorithmSolutionsSimple) {
    val dataset = mutableListOf<Double>()
    for (key in fitnessKeys) {
        if(individual.finalIndividualFitness.avgMatchResult.individualValues[key] == null) {
            dataset.add(0.0)
            System.out.println(key + ": 0.0")
            continue
        }
        dataset.add(Math.abs(individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!) / maxValues[key]!!)
        System.out.println(key + ": " + individual.finalIndividualFitness.avgMatchResult.individualValues[key]!!)
    }
    System.out.println("/////////////////////////////")
    datasets.add(dataset)
}

var fitnessKeysIndexed = mutableListOf<Int>()
for (i in 0 until fitnessKeys.size) {
    fitnessKeysIndexed.add(i + 1)
}

var individualValueIdsMapped = mutableListOf<Int>()
var individualMapped = mutableListOf<String>()
var index = 0
for(dataset in datasets) {
    individualValueIdsMapped.addAll(fitnessKeysIndexed)
    individualMapped.addAll(List(fitnessKeys.size) { "Individual " + (index + 1) })
    index++
}

// Build map for visualization
val selectedIndividualsData = mapOf(
    "IndividualValueLabels" to labels,
    "IndividualValueId" to individualValueIdsMapped,
    "Individual" to individualMapped,
    "IndividualValues" to datasets.map { it }.toMutableList().flatten()
)

// Map each fitnessKey to a map 1 -> "fitnessKey1", 2 -> "fitnessKey2", ...
var labelsIndexed = fitnessKeys.mapIndexed { index, s -> index to s }.toMap().mapKeys { it.key + 1 }

letsPlot(selectedIndividualsData) +
        geomArea(flat = true) {   // <-- flat. I.e., do not transform segments to curves
            x = "IndividualValueId"; y = "IndividualValues"
        } +
        geomPoint() { x = "IndividualValueId"; y = "IndividualValues"; color="Individual" } +
        scaleXDiscrete(labels = labelsIndexed) +
        coordPolar()

DamageTakenPenalty: 0.0
MissilesFired: -0.34545454
MissilesFiredAccuracy: -0.4784727
OpponentDestroyedBonus: 0.0
OpponentTrackingBonus: -4.819454
PowerUp_Pickup_Ammo: -0.8181818
PowerUp_Pickup_Health: 0.0
PowerUp_Pickup_Shield: -2.5757544
SectorExploration: -1.5454272
SurvivalBonus: -5.0
/////////////////////////////
DamageTakenPenalty: 32.355362
MissilesFired: -3.3090906
MissilesFiredAccuracy: -41.70126
OpponentDestroyedBonus: -376.01007
OpponentTrackingBonus: -3.019455
PowerUp_Pickup_Ammo: -7.473691
PowerUp_Pickup_Health: -4.727264
PowerUp_Pickup_Shield: -2.0833364
SectorExploration: -3.757591
SurvivalBonus: -3.9622276
/////////////////////////////


<path d="M170.5 100.04545454545456 L177.4057530686736 100.38471243900432 L184.24499996022723 101.39921888068147 L190.95187498838257 103.0792036188671 L197.46178728026769 105.40848748215933 L203.71204282183166 108.36463819363863 L209.642448235472 111.91918640595705 L215.19589047516592 116.03789987671627 L220.31888685632495 120.68111314367508 L224.96210012328373 125.80410952483408 L229.08081359404295 131.357551764528 L232.63536180636137 137.28795717816834 L235.59151251784067 143.53821271973231 L237.9207963811329 150.04812501161743 L239.60078111931853 156.7550000397728 L240.61528756099568 163.5942469313264 L240.95454545454544 170.5 L240.61528756099568 177.4057530686736 L239.60078111931853 184.2449999602272 L237.9207963811329 190.95187498838257 L235.59151251784067 197.46178728026769 L232.63536180636137 203.71204282183166 L229.08081359404298 209.64244823547196 L224.96210012328373 215.19589047516592 L220.31888685632495 220.31888685632492 L215.19589047516592 224.96210012328373 L209.64244823547196 229.08081359404298 L203.71204282183166 232.63536180636137 L197.46178728026769 235.59151251784067 L190.95187498838257 237.9207963811329 L184.24499996022723 239.60078111931853 L177.4057530686736 240.61528756099568 L170.5 240.95454545454544 L163.5942469313264 240.61528756099568 L156.75500003977277 239.60078111931853 L150.04812501161743 237.9207963811329 L143.53821271973231 235.59151251784067 L137.28795717816834 232.63536180636137 L131.35755176452804 229.08081359404298 L125.80410952483408 224.96210012328373 L120.68111314367506 220.31888685632495 L116.03789987671628 215.19589047516595 L111.91918640595703 209.642448235472 L108.36463819363863 203.71204282183166 L105.40848748215936 197.4617872802677 L103.07920361886711 190.95187498838257 L101.39921888068149 184.24499996022723 L100.3847124390043 177.40575306867356 L100.04545454545455 170.5 L100.3847124390043 163.59424693132644 L101.39921888068149 156.75500003977277 L103.0792036188671 150.04812501161746 L105.40848748215936 143.5382127197323 L108.36463819363863 137.28795717816834 L111.91918640595702 131.35755176452804 L116.03789987671627 125.80410952483408 L120.68111314367505 120.68111314367508 L125.80410952483405 116.03789987671627 L131.357551764528 111.91918640595705 L137.28795717816834 108.36463819363863 L143.53821271973226 105.40848748215936 L150.04812501161743 103.0792036188671 L156.75500003977277 101.3992188806815 L163.5942469313264 100.38471243900432 L170.5 100.04545454545456 " stroke="rgb(233,233,233)" stroke-opacity="1.0" stroke-width="1.0" fill="none">
 
 <path d="M170.5 29.590909090909122 L184.3115061373472 30.269424878008635 L197.98999992045444 32.29843776136295 L211.40374997676514 35.658407237734195 L224.42357456053537 40.31697496431866 L236.92408564366332 46.22927638727725 L248.78489647094398 53.3383728119141 L259.89178095033185 61.575799753432534 L270.1377737126499 70.86222628735015 L279.42420024656747 81.10821904966815 L287.6616271880859 92.21510352905605 L294.77072361272275 104.07591435633668 L300.68302503568134 116.57642543946463 L305.3415927622658 129.59625002323486 L308.70156223863705 143.01000007954556 L310.73057512199136 156.68849386265282 L311.4090909090909 170.5 L310.73057512199136 184.31150613734718 L308.70156223863705 197.98999992045444 L305.3415927622658 211.40374997676514 L300.68302503568134 224.42357456053537 L294.77072361272275 236.92408564366332 L287.66162718808596 248.78489647094392 L279.42420024656747 259.89178095033185 L270.1377737126499 270.13777371264985 L259.89178095033185 279.42420024656747 L248.78489647094392 287.66162718808596 L236.92408564366332 294.77072361272275 L224.42357456053537 300.68302503568134 L211.40374997676514 305.3415927622658 L197.98999992045447 308.70156223863705 L184.3115061373472 310.73057512199136 L170.5 311.4090909090909 L156.68849386265282 310.73057512199136 L143.01000007954553 308.70156223863705 L129.5962500232349 305.3415927622658 L116.57642543946463 300.68302503568134 L104.0759143563367 294.77072361272275 L92.2151035290561 287.661627188085

# Selected Individual Details

In [12]:
import org.jetbrains.letsPlot.intern.Plot

// Input params for visualization START
val selectedIndividuals = mutableListOf<SelectedIndividual>()
selectedIndividuals.add(SelectedIndividual(0, 0, 70, 0)) // Individual at position
selectedIndividuals.add(SelectedIndividual(0, 0, 82, 0)) // Individual at position
// Input params for visualization END

// Prepare data for visualization
var selectedGPAlgorithmSolutionsSimple = mutableListOf<GPProgramSolutionSimple>()

// Get selected individuals
for (selectedIndividual in selectedIndividuals){
    // Prepare data
    val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

    // TODO Remove this
    val avgGenIndividualRatings = mutableListOf<Double>()

    val bestGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val worstGenIndividualAvgFitnesses = mutableListOf<GPProgramSolutionSimple>()
    val avgGenIndividualAvgFitnesses = mutableListOf<Double>()

    prepareData(selectedIndividual.selectedConfiguration, selectedIndividual.selectedRun, bestGenIndividualsRating, worstGenIndividualsRating, avgGenIndividualRatings, bestGenIndividualAvgFitnesses, worstGenIndividualAvgFitnesses, avgGenIndividualAvgFitnesses)

    var individual: GPProgramSolutionSimple?;
    if(selectedIndividual.bestWorstIndividual == 0) {
        individual = bestGenIndividualAvgFitnesses[selectedIndividual.selectedGeneration]
    }
    else {
        individual = worstGenIndividualAvgFitnesses[selectedIndividual.selectedGeneration]
    }

    selectedGPAlgorithmSolutionsSimple.add(individual!!)
}

var plots = mutableListOf<Plot>()

var index = 0
for (individual in selectedGPAlgorithmSolutionsSimple) {
    var nodeCountsData = mapOf(
        "NodeName" to individual.nodeCounts.keys.toList(),
        "NodeCount" to individual.nodeCounts.values.toList()
    )

    val blankTheme = theme(line=elementBlank(), axis=elementBlank())
    val (w, h) = 800 to 400
    val p = letsPlot(nodeCountsData) + ggsize(w,h) + blankTheme

    val plot = p + geomPie(stat = Stat.identity,
            size = 20, stroke = 1, color = "white", hole = 0.5) { slice = "NodeCount"; fill = "NodeName" } +
            blankTheme +
            scaleFillBrewer(palette = "Set1") + ggtitle("Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId)

    System.out.println(
        "Individual: " + "(Conf: " + selectedIndividuals[index].selectedConfiguration + " Run: " + selectedIndividuals[index].selectedRun + " Gen: " + selectedIndividuals[index].selectedGeneration + " IndividualID: " + individual.individualId + ")\nChangesCount: " + individual.changesCount + "\nFunctionNodesCount: " + individual.functionNodes + "\nTerminalNodesCount: " + individual.terminalNodes + "\nTreeSize: " + individual.treeSize + "\nTreeDepth: " + individual.treeDepth + "\nFitness: " + (individual.finalIndividualFitness.additionalValues["Rating"].toString() + ", " + individual.finalIndividualFitness.additionalValues["StdDeviation"]) + "\nIndividualAvgFitness: " + sumIndividualValues(individual.finalIndividualFitness.avgMatchResult) + "\nIndividualAvgFitnessValues: " + individual.finalIndividualFitness.avgMatchResult.individualValues.map { it.key + ": " + it.value }.toString()
        + "\n\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"
    )

    index++

    plots.add(plot)
}

gggrid(plots, ncol = 2)

Individual: (Conf: 0 Run: 0 Gen: 70 IndividualID: 13278)
ChangesCount: 26
FunctionNodesCount: 36
TerminalNodesCount: 39
TreeSize: 75
TreeDepth: 8
Fitness: -39.58899, 2.8713295
IndividualAvgFitness: -277.10125294
IndividualAvgFitnessValues: [OpponentDestroyedBonus: -230.6277, PowerUp_Pickup_Health: -0.90909094, SectorExploration: -2.9999819, SurvivalBonus: -4.948136, DamageTakenPenalty: 4.737909, OpponentTrackingBonus: -2.5670907, PowerUp_Pickup_Ammo: -4.4790454, PowerUp_Pickup_Shield: -1.9264091, MissilesFired: -1.3090909, MissilesFiredAccuracy: -32.072617]
\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\
Individual: (Conf: 0 Run: 0 Gen: 82 IndividualID: 13278)
ChangesCount: 36
FunctionNodesCount: 39
TerminalNodesCount: 47
TreeSize: 86
TreeDepth: 9
Fitness: -49.117485, 2.8582563
IndividualAvgFitness: -413.6886235999999
IndividualAvgFitnessValues: [OpponentDestroyedBonus: -376.01007, PowerUp_Pickup_Health: -4.727264, SectorExploration: -3.757591, SurvivalBonus: -3.9622276, DamageTakenP

Conf: 0 Run: 0 Gen: 70 IndividualID: 13278 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Conf: 0 Run: 0 Gen: 82 IndividualID: 13278 
 
 
 
 
 
 
 
 
 NodeName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RayHitObject 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Inverter 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Sequencer 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ShieldLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rotate 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 HealthLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AmmoLevelBellow 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MoveForward 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Shoot 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Selector

# Genetic Diversity Graph
Genetic diversity of the population based on the avg fitness and node counts

### Calculation of genetic diversity between 2 individuals
1. Difference in avg fitness -> df
2. Difference in node counts -> dn
3. Normalize both values and multiply by the weights -> diversity = w1 * df_norm +  w2 * dn_norm


In [17]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
val w1 = 0.5
val w2 = 0.5
// Input params for visualization END

val bestGenIndividualsRating = mutableListOf<GPProgramSolutionSimple>()

prepareData(selectedConfiguration, selectedRun, bestGenIndividualsRating, null, null, null, null, null)

val gpAlgorithmRunGeneticDiversity = mutableListOf<List<Double?>>()

var diversity = calculateDiversity(bestGenIndividualsRating[82], bestGenIndividualsRating[10], w1, w2)

for(i in 0 until bestGenIndividualsRating.size) {
    var gpAlgorithmGenGeneticDiversity = mutableListOf<Double?>()
    for (j in 0 until bestGenIndividualsRating.size) {
        if(j < i){
            var diversity = calculateDiversity(bestGenIndividualsRating[i], bestGenIndividualsRating[j], w1, w2)
            if(diversity > 1.0 || diversity < 0.0) {
                throw Exception("Diversity out of bounds: " + diversity + ", i: " + i + ", j:" + j)
            }
            gpAlgorithmGenGeneticDiversity.add(diversity)
        }
        else {
            gpAlgorithmGenGeneticDiversity.add(null)
        }
    }
    gpAlgorithmRunGeneticDiversity.add(gpAlgorithmGenGeneticDiversity)
}

var individuals = mutableListOf<Int>()
val generations = mutableListOf<String>()

for (i in 0 until gpAlgorithmRunGeneticDiversity.size) {
    generations.addAll(List(gpAlgorithmRunGeneticDiversity[i].size){(i + 1).toString()})

    for (i in 0 until gpAlgorithmRunGeneticDiversity[i].size) {
        individuals.add(i + 1)
    }
}

val df = dataFrameOf(
    "individuals" to individuals,
    "generations" to generations,
    "diversity" to gpAlgorithmRunGeneticDiversity.flatten()
)

val plot = letsPlot(df.toMap()) { x = "individuals"; y = "generations"; fill = "diversity" } + geomTile() + ggtitle("Genetic Diversity Graph")

plot

<rect fill="#808080

# Master Tournament Graph
Tournament between generation best individuals

In [ ]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

val n = 3
val Generation = mutableListOf<Int>()
val Rating = mutableListOf<Double>()

val masterTournamentGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].masterTournamentGraphData;
for (i in 0 until masterTournamentGraphData.size) {
    Generation.addAll(List(n) {i + 1})
    val rating = Math.abs(masterTournamentGraphData[i].finalIndividualFitness.additionalValues["Rating"]!!)
    Rating.addAll(listOf(rating - (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!), rating, rating + (2*masterTournamentGraphData[i].finalIndividualFitness.additionalValues["StdDeviation"]!!)))
}

val data = mapOf<String, Any>(
    "Generation" to Generation,
    "Rating" to Rating
)

letsPlot(data) { x = "Generation"; y = "Rating"}  + geomBoxplot(whiskerWidth = 0.2) + ggtitle("Convergence graph (Tournament between generation best individuals)")

# Convergence graph
Best individual in the final generation is matched with every best individual from previous generations

In [ ]:
// Input params for visualization START
val selectedConfiguration = 0
val selectedRun = 0
// Input params for visualization END

val convergenceGraphData = progressData.multiConfigurationProgressData[selectedConfiguration].multiRunProgressData[selectedRun].convergenceGraphData;

var generations = mutableListOf<String>()
var fitnesses = mutableListOf<Float>()

for (individualMatch in convergenceGraphData.finalIndividualFitness.individualMatchResults){
    generations.add(individualMatch.opponentsIDs[0].toString())
    fitnesses.add(individualMatch.value)
}

// Display data
val df = dataFrameOf(
    "Generation" to generations,
    "Fitness" to fitnesses
)

val plot = letsPlot(df.toMap()) { x = "Generation"; y = "Fitness"} + geomLine() + geomPoint(shape = 20) + ggtitle("Convergence graph")

plot